In [1]:
from mimicfouretl.data_insights import display_datasets
from mimicfouretl.feature_engineering import *
import mimicfouretl.bigquery_utils as bq

In [2]:
display_datasets()

Dropdown(description='Dataset:', options=('hosp/provider', 'hosp/services', 'hosp/d_icd_procedures', 'hosp/pre…

Output()

In [ ]:
bq.set_credentials_file('../bq_credentials/client_secret.json')
bq.set_project_id('micro-vine-412020')
client = bq.get_client(use_service_account_auth=True)
spark = bq.get_spark_session(client, use_service_account_auth=True)

In [ ]:
results = get_item_frequency(spark, column_name='caregiver_id', dataset='mimic_iv.icu_outputevents')
bq.display_sampled_df(results)

In [ ]:
results = get_temporal_trends(spark=spark, 
                              item_id='226559', 
                              column_name='itemid', 
                              date_column='charttime', 
                              dataset='mimic_iv.icu_outputevents',
                              limit=10)
bq.display_sampled_df(results)

In [ ]:
results = get_outcomes_by_item(spark=spark, 
                              item_id='227074', 
                              item_column='itemid',
                              item_dataset='mimic_iv.icu_inputevents',
                              outcome_column='value',
                              outcome_dataset='mimic_iv.icu_outputevents')
bq.display_sampled_df(results)

In [ ]:
results = get_abnormal_item_analysis(spark=spark, 
                              item_id=51200, 
                              item_column='itemid',
                              value_column='value',
                              bounds={'lower': '0.5'},
                              dataset='mimic_iv.hosp_labevents')
bq.display_sampled_df(results)

In [ ]:
results = get_provider_activity_analysis(spark=spark, 
                                         provider_id='P60CC5', 
                                         dataset_columns={'mimic_iv.hosp_microbiologyevents':
                                                                {'provider': 'order_provider_id', 
                                                                 'activity':'spec_type_desc'}})
bq.display_sampled_df(results)

In [ ]:
results = get_co_occurrence_analysis(spark=spark, 
                                     dataset='mimic_iv.hosp_labevents',
                                     primary_column='priority',
                                     secondary_column='comments')
bq.display_sampled_df(results)

In [ ]:
results = calculate_event_to_death_interval(spark=spark, 
                                     event_date_column='charttime',
                                     event_dataset='mimic_iv.hosp_labevents')
bq.display_sampled_df(results)

24/03/24 16:56:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 906765 ms exceeds timeout 120000 ms
24/03/24 16:56:48 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/24 16:56:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$